# Medical Condition Prediction

 __Problem Statement__
 >The project will aim in predicting wether a person has or has not been diagnosed with one of three medical conditions (B, D,or G). The problem is a binary classification because we expect the outcome to be class1 or class 0
 
__Objective__
>The objective of this project is to create a model that can accurately predict whether a subject has or has not been diagnosed with one of three medical conditions

# Import the necessary libraries

In [462]:
import  pandas as pd
import  pyspark.sql.functions as F
from  pyspark.sql import SparkSession
from  pyspark.sql.functions import isnan, when, count, col
from  pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from  pyspark.ml.classification import RandomForestClassifier,LogisticRegression
from  pyspark.ml.evaluation import BinaryClassificationEvaluator
from  pyspark.ml import Pipeline
from  pyspark.sql.functions import col

 # Create a SparkSession

In [463]:
spark = SparkSession.builder.appName("MedicalConditionPrediction_Netscribes")\
        .getOrCreate()
sqlContext = SparkSession(spark)
spark.sparkContext.setLogLevel("ERROR")

In [464]:
spark

# Load the training data

In [465]:
train_data = spark.read.csv( "train.csv", header=True,  inferSchema=True )

In [466]:
train_data.printSchema( )

root
 |-- Id: string (nullable = true)
 |-- AB: double (nullable = true)
 |-- AF: double (nullable = true)
 |-- AH: double (nullable = true)
 |-- AM: double (nullable = true)
 |-- AR: double (nullable = true)
 |-- AX: double (nullable = true)
 |-- AY: double (nullable = true)
 |-- AZ: double (nullable = true)
 |-- BC: double (nullable = true)
 |-- BD : double (nullable = true)
 |-- BN: double (nullable = true)
 |-- BP: double (nullable = true)
 |-- BQ: double (nullable = true)
 |-- BR: double (nullable = true)
 |-- BZ: double (nullable = true)
 |-- CB: double (nullable = true)
 |-- CC: double (nullable = true)
 |-- CD : double (nullable = true)
 |-- CF: double (nullable = true)
 |-- CH: double (nullable = true)
 |-- CL: double (nullable = true)
 |-- CR: double (nullable = true)
 |-- CS: double (nullable = true)
 |-- CU: double (nullable = true)
 |-- CW : double (nullable = true)
 |-- DA: double (nullable = true)
 |-- DE: double (nullable = true)
 |-- DF: double (nullable = true)
 |-- D

# Load  the  testing  data

In [467]:
test_data = spark.read.csv( "test.csv", header=True, inferSchema=True )

In [468]:
test_data.printSchema( )

root
 |-- Id: string (nullable = true)
 |-- AB: double (nullable = true)
 |-- AF: double (nullable = true)
 |-- AH: double (nullable = true)
 |-- AM: double (nullable = true)
 |-- AR: double (nullable = true)
 |-- AX: double (nullable = true)
 |-- AY: double (nullable = true)
 |-- AZ: double (nullable = true)
 |-- BC: double (nullable = true)
 |-- BD : double (nullable = true)
 |-- BN: double (nullable = true)
 |-- BP: double (nullable = true)
 |-- BQ: double (nullable = true)
 |-- BR: double (nullable = true)
 |-- BZ: double (nullable = true)
 |-- CB: double (nullable = true)
 |-- CC: double (nullable = true)
 |-- CD : double (nullable = true)
 |-- CF: double (nullable = true)
 |-- CH: double (nullable = true)
 |-- CL: double (nullable = true)
 |-- CR: double (nullable = true)
 |-- CS: double (nullable = true)
 |-- CU: double (nullable = true)
 |-- CW : double (nullable = true)
 |-- DA: double (nullable = true)
 |-- DE: double (nullable = true)
 |-- DF: double (nullable = true)
 |-- D

# Load the Greeks data as metadata

In [469]:
metadata = spark.read.csv("greeks.csv", header=True, inferSchema=True)


In [470]:
metadata.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Alpha: string (nullable = true)
 |-- Beta: string (nullable = true)
 |-- Gamma: string (nullable = true)
 |-- Delta: string (nullable = true)
 |-- Epsilon: string (nullable = true)



# Join the metadata with the training data and test dataset

In [471]:
train_data_and_metadata = train_data.join(greeks_data, on='Id')
#test_data_and_metadata = test_data.join(greeks_data, on='Id')

In [472]:
train_data_and_metadata.printSchema()

root
 |-- Id: string (nullable = true)
 |-- AB: double (nullable = true)
 |-- AF: double (nullable = true)
 |-- AH: double (nullable = true)
 |-- AM: double (nullable = true)
 |-- AR: double (nullable = true)
 |-- AX: double (nullable = true)
 |-- AY: double (nullable = true)
 |-- AZ: double (nullable = true)
 |-- BC: double (nullable = true)
 |-- BD : double (nullable = true)
 |-- BN: double (nullable = true)
 |-- BP: double (nullable = true)
 |-- BQ: double (nullable = true)
 |-- BR: double (nullable = true)
 |-- BZ: double (nullable = true)
 |-- CB: double (nullable = true)
 |-- CC: double (nullable = true)
 |-- CD : double (nullable = true)
 |-- CF: double (nullable = true)
 |-- CH: double (nullable = true)
 |-- CL: double (nullable = true)
 |-- CR: double (nullable = true)
 |-- CS: double (nullable = true)
 |-- CU: double (nullable = true)
 |-- CW : double (nullable = true)
 |-- DA: double (nullable = true)
 |-- DE: double (nullable = true)
 |-- DF: double (nullable = true)
 |-- D

In [473]:
pd.DataFrame(train_data_and_metadata.take(5), columns=train_data_and_metadata.columns).head(5)

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,GF,GH,GI,GL,Class,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,2003.810319,22.136229,69.834944,0.120343,1,B,C,G,D,3/19/2019
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,27981.562750,29.135430,32.131996,21.978000,0,A,C,M,B,Unknown
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,13676.957810,28.022851,35.192676,0.196941,0,A,C,M,B,Unknown
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,2094.262452,39.948656,90.493248,0.155829,0,A,C,M,B,Unknown
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8524.370502,45.381316,36.262628,0.096614,1,D,B,F,B,3/25/2020


In [474]:
pd.DataFrame(test_data_and_metadata.take(5), columns=test_data_and_metadata.columns).head(5)

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,GE,GF,GH,GI,GL,Alpha,Beta,Gamma,Delta,Epsilon


In [475]:
print(train_data_and_metadata.columns)


['Id', 'AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB', 'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL', 'Class', 'Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon']


> some columns are not important during the model train. such columns will be dropped

In [476]:
filtered_data=['AB', 'AF', 'AH', 'AM', 'AR', 'AX',
               'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP', 'BQ', 
               'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH',
               'CL', 'CR', 'CS', 'CU', 'CW ', 'DA', 'DE',
               'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 
               'DY', 'EB', 'EE', 'EG', 'EH', 'EJ', 'EL',
               'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
               'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 
               'GL']+ [col for col in train_data_and_metadata.columns if col.startswith("Alpha")] + ["Class"] 
train_data = train_data_and_metadata.select(*filtered_data)

In [477]:
pd.DataFrame(train_data.take(5), columns=train_data.columns).head(5)



,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FR,FS,GB,GE,GF,GH,GI,GL,Alpha,Class
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,B,1
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,A,0
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,A,0
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,A,0
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,D,1


In [478]:
filtered_test=['AB', 'AF', 'AH', 'AM', 'AR', 'AX',
               'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP', 'BQ', 
               'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH',
               'CL', 'CR', 'CS', 'CU', 'CW ', 'DA', 'DE',
               'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 
               'DY', 'EB', 'EE', 'EG', 'EH', 'EJ', 'EL',
               'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
               'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 
               'GL'] + [col  for  col  in  test_data.columns if col.startswith("Alpha")]
test_data = test_data_and_metadata.select(*filtered_test)

# Data cleaning and pre processing
> In this section we will check for null values in the dataset,and drop such rows if they exist

> Also in the schema description of train_data_with_metadata
we saw that alpha column is a string we have to convert it into numeric using using StringIndexer

In [479]:
train_data = train_data.dropna()

__Preprocess the categorical feature "Alpha" and "EJ" using StringIndexer__

In [480]:
# Delete the AlphaIndex column
train_data = train_data.drop("Alpha_index")
train_data = train_data.drop("EJ_index")


In [481]:
indexer_ej = StringIndexer(inputCol="EJ", outputCol="EJ_index").setHandleInvalid("keep")
indexer_alpha = StringIndexer(inputCol="Alpha", outputCol="Alpha_index").setHandleInvalid("keep")


train_data = indexer_ej.fit(train_data).transform(train_data)
train_data = indexer_alpha.fit(train_data).transform(train_data)


In [482]:
pd.DataFrame(train_data.take(5), columns=train_data.columns).head(5)

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,GB,GE,GF,GH,GI,GL,Alpha,Class,EJ_index,Alpha_index
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,B,1,0.0,1.0
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,A,0,1.0,0.0
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,A,0,0.0,0.0
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,A,0,0.0,0.0
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,D,1,0.0,3.0


# combine input features with VectorAssembler

In [483]:
assembler = VectorAssembler(inputCols=['AB', 'AF', 'AH', 'AM', 'AR', 'AX',
               'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP', 'BQ', 
               'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH',
               'CL', 'CR', 'CS', 'CU', 'CW ', 'DA', 'DE',
               'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 
               'DY', 'EB', 'EE', 'EG', 'EH', 'EL',
               'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
               'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 
               'GL', "EJ_index","Alpha_index"], outputCol="features")

In [484]:
train_data = assembler.transform(train_data)

# Split  the  data  into  training  and validation  sets

In [485]:
train_set, val_set = train_data.randomSplit([0.7, 0.3], seed=42)

__use Random forest  algorithm__

In [486]:
rf = RandomForestClassifier(labelCol="Class", featuresCol="features", numTrees=100)
model = rf.fit(train_set)

__Make prediction using the validation data_set__

In [487]:
predictions = model.transform(val_set)

In [488]:
evaluator = BinaryClassificationEvaluator(labelCol="Class")
auc = evaluator.evaluate(predictions)

__Make predictions on the test data__

In [491]:
test_data = test_data.join(greeks_data, on='Id', how="left")


AnalysisException: [UNRESOLVED_USING_COLUMN_FOR_JOIN] USING column `Id` cannot be resolved on the left side of the join. The left-side columns: [`AB`, `AF`, `AH`, `AM`, `AR`, `AX`, `AY`, `AZ`, `BC`, `BD `, `BN`, `BP`, `BQ`, `BR`, `BZ`, `CB`, `CC`, `CD `, `CF`, `CH`, `CL`, `CR`, `CS`, `CU`, `CW `, `DA`, `DE`, `DF`, `DH`, `DI`, `DL`, `DN`, `DU`, `DV`, `DY`, `EB`, `EE`, `EG`, `EH`, `EJ`, `EL`, `EP`, `EU`, `FC`, `FD `, `FE`, `FI`, `FL`, `FR`, `FS`, `GB`, `GE`, `GF`, `GH`, `GI`, `GL`].